In [1]:
#Root 경로로 이동
%cd ..


d:\MyGit\BrainMR_MCI


In [2]:
from model import generate_model
import os
import csv
import torch
import torch.nn as nn
import torch.optim as optim
#from loss import ConRegGroupLoss
import dataloader
import utils
import pandas as pd
from torch.utils.data import DataLoader
from models import resnet


In [3]:
config = utils.load_config()
df_dataset = pd.read_csv(config['PATH_DATASET_CSV'])
X_train,X_test,y_train,y_test = dataloader.dataset_split(df_dataset)
#y_train = y_train.str.replace('MCI','1').replace('CN','0').replace('AD','2')
#y_test = y_test.str.replace('MCI','1').replace('CN','0').replace('AD','2')

In [4]:
traindata=dataloader.MRIDataset(X_train,y_train)
testdata=dataloader.MRIDataset(X_test,y_test)

train_dataloader = DataLoader(traindata, batch_size=1, shuffle=False)
test_dataloader  = DataLoader(testdata, batch_size=1, shuffle=False)

In [5]:
print('X_train : ',len(X_train))
print('X_test : ',len(X_test))

X_train :  792
X_test :  199


In [6]:
model = resnet.resnet10(
    num_classes=3,
    shortcut_type='B',
    sample_size=112,
    sample_duration=16)

d:\MyGit\BrainMR_MCI\models\resnet.py:145: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


In [7]:
model

ResNet(
  (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
  (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
      (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, 

In [8]:
device = torch.device('cpu')

#model, parameters = generate_model(opt)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr= 0.0001, betas=(0.5, 0.999))
criterion_clf = nn.CrossEntropyLoss().to(device)
criterion_reg = nn.SmoothL1Loss(reduction='mean').to(device)

In [9]:
train_logger = utils.Logger(
    os.path.join('./log/', 'train.log'),
    ['epoch', 'loss', 'lr'])
train_batch_logger = utils.Logger(
    os.path.join('./log/', 'train_batch.log'),
    ['epoch', 'batch', 'iter', 'loss', 'lr'])

In [10]:
epoch = 0
i= 0

model.train(True)
for inputs, labels in train_dataloader:
    i+=1
    inputs, labels = inputs.to(device), labels.to(device)
    model.zero_grad()
    clf_output = model(inputs)
    clf_loss = criterion_clf(clf_output, labels)
    print('loss :',clf_loss)
    clf_loss.backward()
    optimizer.step()
    train_batch_logger.log({
            'epoch': epoch,
            'batch': i + 1,
            'iter': (epoch - 1) * len(train_dataloader) + (i + 1),
            'loss': clf_loss.item(),
            'lr': optimizer.param_groups[0]['lr']
        })



KeyboardInterrupt: 

In [ ]:
def train_model_epoch(self):
    self.model.train(True)
    for inputs, labels, demors in self.train_dataloader:
        inputs, labels, demors = inputs.cuda(), labels.cuda(), demors.cuda()
        self.model.zero_grad()
        loss = torch.tensor(0.0, requires_grad=True).cuda()
        clf_output, reg_output, per_loss = self.model(inputs)
        clf_loss = self.criterion_clf(clf_output, labels)
        con_reg_group_loss = self.get_con_reg_group_loss.apply(reg_output, demors, self.frequency_dict, labels)
        loss = loss + clf_loss + reg_loss + torch.mean(con_reg_group_loss) + torch.mean(per_loss)
        loss.backward()
        self.optimizer.step()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr= 0.0001, betas=(0.5, 0.999))

criterion_clf = nn.CrossEntropyLoss(weight=torch.Tensor([1, 3])).to('mps')
criterion_reg = nn.SmoothL1Loss(reduction='mean').to('mps')

optimal_valid_matrix = [[0, 0], [0, 0]]
optimal_valid_metric = 0
optimal_epoch = 0

epoch = 0
train_epoch = 300

while epoch < train_epoch:
    self.train_model_epoch()

    print('{}th epoch validation confusion matrix:'.format(self.epoch), valid_matrix)
    print('eval_metric:', "%.4f" % self.eval_metric(valid_matrix), 'and mean squared error ', valid_mse)
    self.save_checkpoint(valid_matrix, valid_mse)
    self.epoch += 1
